# 3. Wolfram Cloud™ Integration with Python Notebooks

Continuation of Wolfram Language™ for Python users. This notebook contains examples of:
- setting up and utilising the Wolfram Cloud™ for evaluations.
- deploying and accessing Wolfram APIs. 
- communication between languages.

Before you start, you will need to set up a cloud authentication key:

https://reference.wolfram.com/language/WolframClientForPython/docpages/basic_usages.html#wolfram-cloud-interactions

In [1]:
from wolframclient.evaluation import SecuredAuthenticationKey, WolframCloudSession
sak = SecuredAuthenticationKey(
    '0CRpn5DTG6P7Dhifa/6oZTlaB+/g0UPNH6cRCDVm814=',
    'q86IUU+nz8bAh64mK/mwYhzciwv0CNK76jkkcOqxJwA=')

In [2]:
session = WolframCloudSession(credentials=sak)
session.start()
session.authorized()

True

The created cloud session can then be used just as with a local session:

In [3]:
from wolframclient.language import wl, wlexpr
session.evaluate(wl.Range(3))

PackedArray([1, 2, 3], dtype=int8)

Note however that session.evaluate now functions as CloudEvaluate, so no information or definitions will persist:

In [4]:
session.evaluate(wlexpr('x = 1'))

1

In [5]:
session.evaluate(wlexpr('x'))

Global`x

Functions therefore must be defined using **session.function**:

In [6]:
wl_str_reverse = session.function(wl.StringReverse)

In [7]:
wl_str_reverse("abcd")

'dcba'

## API Cloud Functions

Deployed APIs are one of the most versatile ways of utilising the Wolfram Language. They can be called from within browsers, as well as other languages:

In [ ]:
CloudConnect["MyWolframID", "myPassword"]
api = APIFunction[{"x" -> "Integer"},
    #x^2 &
]
CloudDeploy[api, CloudObject["api/private/xsquared"]]
SetPermissions[
    CloudObject["api/private/xsquared"],
    SecuredAuthenticationKeys["pythonclientlibrary"] -> {"Read", "Execute"}
]

Example: Pre-built classification function training on *Titanic* passenger data: 

In [9]:
cloud = WolframCloudSession()

api = ('mjames', 'api/public/titanic')
result = cloud.call(api, {'class': '1st', 'age': 43, 'gender': 'male'})

result.get().decode("utf-8") 

'"died"'

**WolframAPICall** represents a convenient way to perform API calls by setting up parameters separately:

In [11]:
from wolframclient.evaluation import WolframAPICall
call = WolframAPICall(cloud, ('mjames', 'api/public/titanic'))
call.set_parameter('class', '1st')
call.set_parameter('age', 43)
call.set_parameter('gender', 'male')
result = call.perform()
result.get().decode("utf-8") 

'"died"'